# Alignment and Preprocessing

Once the data is made available as detailed in [Data Ingestion](./01_Data_Ingestion.ipynb), the next step is to ensure the data has been appropriately reshaped and aligned across data sources for consumption by the machine learning pipeline (or other analysis or simulation steps).

We'll be aggregating data across several years, using the same Landsat images used in the [Walker Lake](../topics/Walker_Lake.ipynb) example. See that topic for more work on calculating the difference between the water levels over time.

In [ ]:
import intake
import numpy as np
import xarray as xr

import holoviews as hv

import cartopy.crs as ccrs
import geoviews as gv

import hvplot.xarray

hv.extension('bokeh', width=80)

## Recap: Loading data

In [ ]:
cat = intake.open_catalog('../catalog.yml')
l5_da = cat.l5().read_chunked()
l5_da

In [ ]:
l8_da = cat.l8().read_chunked()
l8_da

We can use this EPSG value shown above under the ``crs`` key to create a Cartopy coordinate reference system that we will be using later on in this notebook:

In [ ]:
crs=ccrs.epsg(32611)

## Preprocessing
The first step in processing data is to remove the missing values. In this case xarray self-reports the values assigned to `nodatavals`. We can use this information to set the missing values to `NaN`.

In [ ]:
l5_da = l5_da.where(l5_da > l5_da.nodatavals[0])
l8_da = l8_da.where(l8_da > l8_da.nodatavals[0])

We can make sure that no more -9999s show up in the data, by calculating the minimum value in each DataArray as follows:

In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings('ignore', r'All-NaN slice encountered')
    l5_min = l5_da.min().compute()
    l8_min = l8_da.min().compute()
(l5_min, l8_min)

**NOTE:** These operations take a non-trivial amount of time because they require that the data actually be loaded. 

## Compute NDVI

Now we will calculate the [NDVI](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index) (vegetation index) for each of these image sets.

In [ ]:
NDVI_1988 = (l5_da.sel(band=5) - l5_da.sel(band=4)) / (l5_da.sel(band=5) + l5_da.sel(band=4))
NDVI_1988

In [ ]:
NDVI_2017 = (l8_da.sel(band=5) - l8_da.sel(band=4)) / (l8_da.sel(band=5) + l8_da.sel(band=4))
NDVI_2017

**NOTE:** To calculate NDVI we have selected based on the band label. In `xarray` you can also select by index.

In [ ]:
# Exercise: Try selecting by index like l8_da[0, 0, 0] or l8_da[0:1, 0:100, 0:100]

## Combine data from overlapping grids

These two sets of Landsat bands cover roughly the same area but were taken in 1988 and 2017. While they have the same resolution (30m), they have different numbers of grid cells and different x and y offsets. We can see that by printing the first `x` value for each year and seeing that they are not equivalent.

In [ ]:
NDVI_1988.x[0].item(), NDVI_2017.x[0].item()

### Treating year as DataArray name

XArray allows arbitrary data to be combined into one `xr.Dataset`, though if the underlying grids are not aligned, then the shape of the dataset grows to be the union of the dimensions on each array. Here we'll combine the data from different years, providing the year as the name of the underlying DataArray:

In [ ]:
ds = xr.Dataset({'1988': NDVI_1988, '2017': NDVI_2017})
ds

We can quickly sample one point, such as the center of Walker lake: latitude 38.6942° N, longitude 118.7081° W. First convert the lat, lon (PlateCarree) values to the coordinate reference system of the data (using the `crs` var we defined above):

In [ ]:
x_center, y_center = crs.transform_point(-118.7081, 38.6942, ccrs.PlateCarree())

Then we'll select the data point nearest to this point, and use `.compute()` to actually load that data using `dask`:

In [ ]:
ds.sel(x=x_center, y=y_center, method='nearest').compute()

In [ ]:
# Exercise: Use shift-tab completion to explore different interpolation methods for sel, and try using a different one ('backfill')

## Select region of interest

We'll use the area around the central point as the Region of Interest (ROI). In this case we'll use a 30 km box around the center point.

In [ ]:
buffer = 1.5e4

In [ ]:
xmin = x_center - buffer
xmax = x_center + buffer
ymin = y_center - buffer
ymax = y_center + buffer

In [ ]:
ROI = ds.sel(x=slice(xmin, xmax), y=slice(ymin, ymax))
ROI

Since the data are on the same coordinate system, when these data are visualized, the plots can be linked. For instance, if you pan one of the plots, the other will pan as well:

In [ ]:
args = dict(cmap='viridis', width=500, height=500, crs=crs, rasterize=True)

NDVI_1988_p = ROI['1988'].hvplot(clim=(-3, 1), **args).relabel('1988')
NDVI_2017_p = ROI['2017'].hvplot(clim=(-3, 1), **args).relabel('2017')

(NDVI_1988_p + NDVI_2017_p).options(shared_axes=True)

In [ ]:
# Exercise: Pan around these plots, zoom in and out, reset.

We can also subtract the two ROIs, to visualize the difference between 2017 and 1988:

In [ ]:
(ROI['2017'] - ROI['1988']).hvplot(crs=crs, clim=(-2, 2), rasterize=True,
                                   width=600, height=500, cmap='coolwarm').relabel('Difference in NDVI')

#### Treating year as coords

Another way to join the data from the two different years, is by treating the years as coordinates. This approach is more logically sound, but sometimes global attribute data can be lost. For instance if the `crs` were different on the two dataarrays then that attribute would not appear on the output. To work around this, it is sometimes helpful to assign any `attrs` that differ to `coords`. This would look like `NDVI_2017.assign_coords(crs=NDVI_2017.attrs['crs'])`.

In [ ]:
NDVI_by_year = xr.concat([NDVI_1988, NDVI_2017], dim=xr.DataArray([1988, 2017], dims=('year'), name='year'))
NDVI_by_year

We'll do the same sampling at a point and select a ROI to demonstrate how years as coords differ from years as names.

In [ ]:
NDVI_by_year.sel(x=x_center, y=y_center, method='nearest').compute()

In [ ]:
ROI = NDVI_by_year.sel(x=slice(xmin, xmax), y=slice(ymin, ymax))

**NOTE:** It is simpler to define a series of subplots where the variable that is being iterated over is a coordinate.


In [ ]:
p = ROI.hvplot('x','y', col='year', clim=(-3, 1), dynamic=False, **args)
print(p)

Since the output is a gridspace, we can select a a subplot from the output and alter it in place. For instance, we only really need the colorbar on the second subplot, so we can turn it off for the first one:

In [ ]:
p[1988] = p[1988].options(colorbar=False)

In [ ]:
p

Now we can compute the difference:

In [ ]:
ROI.diff('year').squeeze().hvplot('x', 'y', crs=crs, cmap='coolwarm', width=600, height=500, rasterize=True)

## Aligning data on different grids

Suppose that the two sets of Landsat images are on a different grid, which often happens because of differences in how data is collected and processed. To illustrate how to handle this case, we'll manually shift the 2017 coordinates slightly:

In [ ]:
NDVI_2017['x'] = NDVI_2017.x+15
NDVI_2017['y'] = NDVI_2017.y+15

It is still possible to concatenate the arrays, but the x and y coordinates are now 2 times greater than when the arrays are on the same grid.

In [ ]:
ds = xr.concat([NDVI_1988, NDVI_2017],  dim=xr.DataArray([1988, 2017], dims=('year'), name='year'))
ds

In [ ]:
# Exercise: Use .shape to look at the shape of NDVI_by_year defined above compared to the ds that we just defined

These data can be plotted just as before:

In [ ]:
ROI = ds.sel(x=slice(xmin, xmax), y=slice(ymin, ymax))

In [ ]:
ROI.hvplot('x', 'y', col='year', **args)

Since the years don't share coordinates, the difference is all NaNs

In [ ]:
ROI.diff('year').isnull().all().compute()

To resolve this we can declare a new grid that spans the entire ROI with the same resolution as the original arrays (30m)

In [ ]:
res = 30
x = np.arange(ROI.x.min(), ROI.x.max(), res)
y = np.arange(ROI.y.min(), ROI.y.max(), res)

We will use linear interpolation to calculate the values for each grid cell. This operation is not yet supported for dask arrays, so we'll first load in the data. 

In [ ]:
NDVI_1988.load()
NDVI_2017.load()

Now we can use the new coordinates defined above to interpolate from the input coordinates to the new grid. The options are `nearest` and `linear` with linear being selected by default.

In [ ]:
re_indexed_2017 = NDVI_2017.interp(x=x, y=y)
re_indexed_1988 = NDVI_1988.interp(x=x, y=y)

In [ ]:
# Exercise: Try using linear interpolation instead of nearest. Use tab and shift-tab for help.

In [ ]:
ROI = xr.concat([re_indexed_1988, re_indexed_2017],  dim=xr.DataArray([1988, 2017], dims=('year'), name='year'))
ROI

And now we can plot the difference between the two years:

In [ ]:
# Exercise: Plot the difference between the two years using the same approach as we used above.

### Next:

Now that the data have been aligned and preprocessed appropriately, we can then use them in subsequent steps in a workflow. For instance you might need to learn more about [Resampling](04_Resampling.ipynb). Or you might already be ready for [Machine Learning](05_Machine_Learning.ipynb).